In [ ]:
#Here we make all phylogenetic trees of the investigated Phyla with >500 OTUs

In [1]:
import random
import feather
import os
import matplotlib.pyplot as plt
import subprocess
from Bio.Phylo.PhyloXML import Phylogeny
from Bio import Phylo
import networkx, pylab
from tqdm import tqdm
import math

In [2]:
#function to import the microbeatlas files with taxonbomy and OTU-abudnances
import h5py
import numpy as np
import pandas as pd
from scipy.sparse import csc_matrix

def read_mapdb_h5(h5_path, otu_data=True, meta_data=True):
    """Example file: /mnt/mnemo3/janko/data/microbe_atlas/sruns-otus.97.otutable_plusMetaNoHeader_taxonomy_unmapped.h5"""
    f = h5py.File(h5_path, 'r')
    result_dict = {}

    # read otu data
    if otu_data:
        data_handle = f["otu_table"]
        col_ptr, nzvals, rowindices, otu_index, sample_index = [np.array(data_handle[sub_key]) for sub_key in ['data_colptr', 'data_nzval', 'data_rowval', 'oids', 'sids']]

        ## correct indexing (julia starts at 1)
        col_ptr -= 1
        rowindices -= 1

        otutable_sparse = csc_matrix((nzvals,rowindices,col_ptr), shape=(data_handle["m"][()],data_handle["n"][()]))
        result_dict["otu_data"] = {"otu_table": otutable_sparse, "otu_index": otu_index, "sample_index": sample_index}

    # read meta data
    if meta_data:
        meta_handle = f["meta_data"]
        result_dict["meta_data"] = {sub_key: pd.DataFrame(np.array(meta_handle[sub_key]).T) for sub_key in meta_handle.keys()}

    f.close()

    return result_dict



## Select Phyla, Families etc.

In [3]:
savepath="/mnt/mnemo6/lukas/Python/community_conservatism/taxo_groups/"

In [4]:
#select OTU level here: in This case 99%
f1=read_mapdb_h5("/mnt/mnemo3/janko/data/microbe_atlas/hdf5/v0.2.2/metag_minfilter/samples-otus.99.metag.minfilter.remap.h5")
tax=f1["meta_data"]["taxonomy"]

In [5]:
tax=f1["meta_data"]["taxonomy"]

In [15]:
#select all bacteria and archeael phyla with >500 otus
taxbac=tax[tax[2].str.contains("Archaea|Bacteria")]
taxesb=taxbac[2].str.split(";",expand=True)
#taxesb=tax[2].str.split(";",expand=True)

#Phyla:
#taxesb[1].value_counts()


#Classes:
classs=taxesb[2].value_counts()

#order:3
#family:
fam=taxesb[4].value_counts()
phyla=classs=taxesb[1].value_counts()

In [26]:
# all the phyla with more than 500 OTUs
phylanames=phyla[phyla>500].index


In [23]:
def makedirs(taxnam):
    inp_1="mkdir "+savepath+taxnam
    subprocess.run(inp_1,shell=True)
    inp_2="mkdir "+savepath+taxnam+"/plots"
    subprocess.run(inp_2,shell=True)
    inp_3="mkdir "+savepath+taxnam+"/tree"
    subprocess.run(inp_3,shell=True)
      

In [24]:
for phyl in phylanames:
    makedirs(phyl)

### lets create the phylo tree here

In [36]:
def save_alignment(taxname):
    tax_of_interest=tax[tax[2].str.contains(taxname)]

    taxlist=tax_of_interest[1].tolist()

    #get the alignment files of all representative 16S sequences from MApREF:

    
    if "archaeota" in taxname:
        all_alignment_file=pd.read_csv("/mnt/mnemo6/lukas/Python/evo_treedists/global_alignment/all_archaea_99_alignments.tsv",sep="\t")


        all_alignment_file.index=all_alignment_file["ID"]
    else:
        all_alignment_file=pd.read_csv("/mnt/mnemo6/lukas/Python/evo_treedists/global_alignment/all_99_alignments.tsv",sep="\t")


        all_alignment_file.index=all_alignment_file["ID"]


    #get all the sequences into one file



    #loop through our list of IDs belonging to the tax, then extract the alignment of it and save into file

    alignment_str=""
    c=0
    for ids in taxlist:
        try:
            alignment_str+=">"+ids+"\n"+all_alignment_file.loc[ids,"sequ"]+"\n"
            #alignment_str+=
        except:

            c+=1
    print(str(c)+" not found")

    #save the alignment file:

    align=open(savepath+taxname+"/tree/"+taxname+"_alignment.afa","w")
    align.write(alignment_str)
    align.close()

In [38]:
#making the trees, always using the same model to ensure comparability
def makefasttree(taxname):
    com4="/mnt/mnemo6/lukas/anaconda3/bin/FastTree -nt -gtr -gamma "+savepath+taxname+"/tree/"+taxname+"_alignment.afa > /mnt/mnemo6/lukas/Python/community_conservatism/taxo_groups/"+taxname+"/tree/"+taxname+"_tree.txt" 
    subprocess.run(com4,shell=True)

In [ ]:
for phy in phylanames:
    save_alignment(phy)

In [ ]:
for phy in phylanames:
    makefasttree(phy)

In [52]:
#Get the trees into a different format to harmonize with later results and workflows
def fixtrees(taxname):
    tree_old=open(savepath+taxname+"/tree/"+taxname+"_tree.txt","r")
    treeoldl=tree_old.readlines()
    tree_old.close()

    newtr=[]
    for li in treeoldl:
        newtr.append(li.replace(";","_"))
    newtrstr="".join(newtr)
    newstrstr=newtrstr[:-2]+";"
    new=open(savepath+taxname+"/tree/"+taxname+"_tree_fix.txt","w")
    new.write(newstrstr)
    new.close()

In [53]:
for phy in phylanames:
    fixtrees(phy)

In [43]:
#writes a file with the tree paths
stritr=""
for taxname in phylanames:
    stritr+=savepath+taxname+"/tree/"+taxname+"_tree_fix.txt\n"

ft=open("tree_paths.txt","w")
ft.write(stritr)
ft.close()